In [2]:
import tweepy
import shutil
import jsonpickle
import json
import datetime
import GetOldTweets3 as got

from pandas.io.json import json_normalize

import pandas as pd

In [3]:
# load Twitter API credentials
with open('../twitter_credentials.json') as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_token = info['ACCESS_TOKEN']
    access_secret = info['ACCESS_SECRET']

In [4]:
# Authenticate to Twitter
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

try:
    api.verify_credentials()
    print("Authentication OK")
except:
    print("Error during authentication")

Authentication OK


In [8]:
user = api.me()
print (user.name)
print(api.rate_limit_status()['resources']['search'])

Tee Moore
{'/search/tweets': {'limit': 180, 'remaining': 180, 'reset': 1563900800}}


In [6]:
my_accounts = []
for member in tweepy.Cursor(api.list_members, 'ClydeLazersex', 'evac-route-project').items():
    my_accounts.append(member.screen_name)
# Show list
my_accounts

['GovKemp',
 'NC_Governor',
 'chrisWSOC9',
 'FahertyWSOC9',
 'DCTrafficHater',
 'SCDOTLowCountry',
 'ChathamCountyNC',
 'NWSWilmingtonNC',
 'NWSMoreheadCity',
 'FloridaStorms',
 'NWSGSP',
 'NWSAtlanta',
 'NWSJacksonville',
 'NWSEastern',
 'Glynn_Police',
 'SCDPS_PIO',
 'ChasCoSheriff',
 'NWS',
 'NCPublicSafety',
 'St_Simons',
 'NWSColumbia',
 'GlynnCounty',
 '511Georgia',
 'AllisonWSOC9',
 'NWSCharlestonSC',
 'NHC_Atlantic',
 'TotalTrafficJAX',
 'GADeptofTrans',
 'kenlemonWSOC9',
 'WTOPtraffic',
 'scdhec',
 'fl511_northeast',
 'ChasCountyGov',
 'NCEmergency',
 'ChathamEMA',
 'ActionTraffic',
 'Golden_Isles',
 'SavPolice',
 'cityofsavannah',
 'FLSERT',
 'NCDOT_Triangle',
 'SCEMD',
 'brunswickgapd',
 'GeorgiaPower',
 'GeorgiaAlert',
 'NCSHP',
 'WOKVNews',
 'ActionNewsJax',
 'CityCharleston',
 'wsoctv',
 'wsbtv',
 'GeorgiaEMA',
 'CDCemergency',
 'femaregion4',
 'FEMAregion3',
 'NCDOT',
 'VDEM',
 'jaxdotcom',
 'WRAL',
 'postandcourier',
 'GARedCross',
 'newsobserver']

In [9]:
# code modified from Jasmine Vasandani
# With the help of got.manager, get all the tweets from 01/01/2016 till 04/14/2019
# Store all the collected tweets in DF
max_tweets=3000
closure_list=[]
final_df=pd.DataFrame()
for account in my_accounts:
    tweetCriteria = got.manager.TweetCriteria().setUsername(account).setSince("2016-10-01").setUntil("2016-11-30").setMaxTweets(max_tweets)   
    print(tweetCriteria)
    tweet = got.manager.TweetManager.getTweets(tweetCriteria)
    for i in range(len(tweet)):
        tweet_dict = {}
        tweet_dict['id'] = tweet[i].id
        tweet_dict['username'] = tweet[i].username
        tweet_dict['date'] = tweet[i].date
        tweet_dict['text'] = tweet[i].text
        tweet_dict['hashtags'] = tweet[i].hashtags
        tweet_dict['geo'] = tweet[i].geo
        tweet_dict['type'] = 'official'
        closure_list.append(tweet_dict)


df1=pd.DataFrame(closure_list)
final_df = pd.concat([df1, final_df])
print("Shape of DataFrame:", final_df.shape)

Shape of DataFrame: (43623, 7)


In [10]:
# review the shape of the dataframe
final_df.shape

(43623, 7)

In [15]:
closure_list[5000]

{'id': '789553658539237376',
 'username': '511Georgia',
 'date': datetime.datetime(2016, 10, 21, 19, 47, 16, tzinfo=datetime.timezone.utc),
 'text': 'Accident, I-75 Southbound past Forest Parkway (Exit 237), 2 of 4 left lanes blocked. #ClaytonCounty #511GApic.twitter.com/75N2r1ViZm',
 'hashtags': '#ClaytonCounty #511GApic',
 'geo': '',
 'type': 'official'}

In [12]:
final_df.iloc[300:350].text

300       CLEARED: Accident; I-26 EB, at Exit159.| 8:51P
301    Update: Accident; I-26 WB: at Exit159, no lns ...
302    Accident; I-26 WB: at Exit159, all lns blkd, O...
303    Accident; I-26 EB: at Exit159, no lns clsd, Oc...
304       CLEARED: Accident; I-95 SB, at Exit86A.| 3:24P
305    Update: Accident; I-95 SB: at Exit86A, no lns ...
306    Update: Accident; I-95 SB: at Exit86A, no lns ...
307    CLEARED: Obstruction on roadway; I-526 WB, 1 m...
308    Obstruction on roadway; I-526 WB: 1 mi E of Ex...
309    Accident; I-95 SB: at Exit86A, lft ln clsd, Oc...
310        CLEARED: Accident; I-95 NB, at Exit28.| 1:11P
311    Accident; I-95 NB: at Exit28, no lns clsd, Ocr...
312       CLEARED: Accident; I-95 NB, at Exit28.| 12:56P
313    Update: Accident; I-95 NB: at Exit28, no lns c...
314    Update: Accident; I-95 NB: 2 mi S of Exit38, n...
315    Update: Accident; I-95 NB: 2 mi S of Exit38, u...
316    Accident; I-95 NB: 2 mi S of Exit42, unknown l...
317    CLEARED: Disabled veh; I

In [16]:
# list the columns from the dataframe
final_df.columns

Index(['date', 'geo', 'hashtags', 'id', 'text', 'type', 'username'], dtype='object')

In [17]:
# list the accounts followed and the count of Tweets collected from each account
final_df.groupby(by='username')['username'].count()

username
511Georgia          364
ActionNewsJax      2569
ActionTraffic       527
AllisonWSOC9        153
CDCemergency         59
ChasCoSheriff        71
ChasCountyGov       128
ChathamCountyNC      37
ChathamEMA         1017
CityCharleston       62
FEMAregion3          42
FLSERT              404
FahertyWSOC9        127
FloridaStorms        79
GADeptofTrans       468
GARedCross           23
GeorgiaAlert         33
GeorgiaEMA          227
GeorgiaPower       1329
GlynnCounty          43
Glynn_Police          2
Golden_Isles        288
NCDOT               412
NCDOT_Triangle     2080
NCEmergency         665
NCPublicSafety      611
NCSHP               460
NHC_Atlantic        326
NWS                 233
NWSAtlanta          183
NWSCharlestonSC     607
NWSColumbia         259
NWSEastern          112
NWSGSP              775
NWSJacksonville     252
NWSMoreheadCity     245
NWSWilmingtonNC     221
SCDOTLowCountry    1446
SCDPS_PIO            47
SCEMD               741
SavPolice           311
St_Simo

In [19]:
final_df.to_csv('../data/Tweets_Historical/historic_tweets_07232019.csv', index = False)